# Mosaico 2008

Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Mosaicos Patagonia

2021-08-05 13:07:09 

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [2]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map1 = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)
Map11 = geemap.Map(center = [-45, -70], zoom = 4)
Map12 = geemap.Map(center = [-45, -70], zoom = 4)
Map13 = geemap.Map(center = [-45, -70], zoom = 4)
Map14 = geemap.Map(center = [-45, -70], zoom = 4)
Map15 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'corte.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

In [4]:
base1 = os.path.join(work_dir, 'base_geo.shp')
base = geemap.shp_to_ee(base1)

In [5]:
base2 = os.path.join(work_dir, 'base2.shp')
base2 = geemap.shp_to_ee(base2)
base3 = os.path.join(work_dir, 'base3.shp')
base3 = geemap.shp_to_ee(base3)
base4 = os.path.join(work_dir, 'base4.shp')
base4 = geemap.shp_to_ee(base4)
base5 = os.path.join(work_dir, 'base5.shp')
base5 = geemap.shp_to_ee(base5)
base6 = os.path.join(work_dir, 'base6.shp')
base6 = geemap.shp_to_ee(base6)

In [6]:
geometria_base = os.path.join(work_dir, 'mapa_base_dis.shp') 
geometria_base = geemap.shp_to_ee(geometria_base)

In [7]:
geometria_base2 = os.path.join(work_dir, 'area_base2_dis.shp') 
geometria_base2 = geemap.shp_to_ee(geometria_base2)

## Colors Bars

In [8]:
import geemap.colormaps as cm

In [9]:
# cm.palettes.terrain

In [10]:
color_ndvi = cm.palettes.ndvi

In [11]:
color_ndwi = cm.palettes.ndwi

In [12]:
color_terrain = cm.palettes.terrain

In [13]:
color_glaciar = cm.palettes.PuBuGn_r

In [14]:
color_nubes = cm.palettes.inferno

In [15]:
# cm.list_colormaps()

In [16]:
# cm.plot_colormaps(width=12, height=0.4)

## Funciones

In [17]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [18]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('B11')
    return image.addBands(ndvi)

In [19]:
def addNDVI7(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [20]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [21]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [22]:
def addSAVI(image):
    savi = image.expression(
        '((b4 - b3)/(b4 + b3 + 0.5))*1.5', {
            'b3':image.select('B3'),
            'b4':image.select('B4')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

In [23]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B2','B5']).rename('NDSI')
    return image.addBands(ndsi)

In [24]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMM'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [25]:
def Cloud40(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(40)) 

In [26]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [27]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [28]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [29]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

In [30]:
def find_closest(target_image, image_col, days):
    """Filter images in a collection by date proximity and spatial intersection to a target image.

    Args:
        target_image: An ee.Image whose observation date is used to find near-date images in
          the provided image_col image collection. It must have a 'system:time_start' property.
        image_col: An ee.ImageCollection to filter by date proximity and spatial intersection
          to the target_image. Each image in the collection must have a 'system:time_start'
          property.
        days: A number that defines the maximum number of days difference allowed between
          the target_image and images in the image_col.

    Returns:
        An ee.ImageCollection that has been filtered to include those images that are within the
          given date proximity to target_image and intersect it spatially.
    """

    # Compute the timespan for N days (in milliseconds).
    range = ee.Number(days).multiply(1000 * 60 * 60 * 24)

    filter = ee.Filter.And(
        ee.Filter.maxDifference(range, 'system:time_start', None, 'system:time_start'),
        ee.Filter.intersects('.geo', None, '.geo'))

    closest = (ee.Join.saveAll('matches', 'measure')
        .apply(ee.ImageCollection([target_image]), image_col, filter))

    return ee.ImageCollection(ee.List(closest.first().get('matches')))

In [31]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [32]:
vizParams_m = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.6,
  'gamma': [1, 1, 1]
}

In [33]:
vis_nieve = {
  'bands': ['B6', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

## Colecciones

### 2009

In [34]:
start_date = '2008-10-01'
end_date = '2009-03-31'

l5_2009 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [35]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [36]:
vis_nieve = {
  'bands': ['B5', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [37]:
coleccion2009 = l5_2009.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [38]:
QM_NDVI_2009 = coleccion2009.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2009 = coleccion2009.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2009 = coleccion2009.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2009 = coleccion2009.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2009 = coleccion2009.qualityMosaic('NDSI') # Normalized difference snow index

In [39]:
Map.addLayer(QM_NDVI_2009, vizParams, 'QM de NDVI 2009')
Map.addLayer(QM_NDWI_2009, vizParams, 'QM de NDWI 2009')
Map.addLayer(QM_SAVI_2009, vizParams, 'QM de SAVI 2009')
Map.addLayer(QM_NDGI_2009, vis_nieve, 'QM de NDGI 2009')
Map.addLayer(QM_NDSI_2009, vis_nieve, 'QM de NDSI 2009')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [40]:
c2009_sn = coleccion2009.map(Cloud20)
NDVI_2009_sn = c2009_sn.qualityMosaic('B11')
NDWI_2009_sn = c2009_sn.qualityMosaic('NDWI')
SAVI_2009_sn = c2009_sn.qualityMosaic('SAVI')
NDGI_2009_sn = c2009_sn.qualityMosaic('NDGI')
NDSI_2009_sn = c2009_sn.qualityMosaic('NDSI')

In [41]:
# Map2.addLayer(NDVI_2009_sn, vizParams, 'QMsn de NDVI 2009')
# Map2.addLayer(NDWI_2009_sn, vizParams, 'QMsn de NDWI 2009')
# Map2.addLayer(SAVI_2009_sn, vizParams, 'QMsn de SAVI 2009')
# Map2.addLayer(NDGI_2009_sn, vis_nieve, 'QMsn de NDGI 2009')
# Map2.addLayer(NDSI_2009_sn, vis_nieve, 'QMsn de NDSI 2009')
# Map2

### 2008

In [42]:
start_date = '2007-10-01'
end_date = '2008-03-31'

l5_2008 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [43]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [44]:
vis_nieve = {
  'bands': ['B5', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [45]:
coleccion2008 = l5_2008.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [46]:
QM_NDVI_2008 = coleccion2008.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2008 = coleccion2008.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2008 = coleccion2008.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2008 = coleccion2008.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2008 = coleccion2008.qualityMosaic('NDSI') # Normalized difference snow index

In [47]:
Map.addLayer(QM_NDVI_2008, vizParams, 'QM de NDVI 2008')
Map.addLayer(QM_NDWI_2008, vizParams, 'QM de NDWI 2008')
Map.addLayer(QM_SAVI_2008, vizParams, 'QM de SAVI 2008')
Map.addLayer(QM_NDGI_2008, vis_nieve, 'QM de NDGI 2008')
Map.addLayer(QM_NDSI_2008, vis_nieve, 'QM de NDSI 2008')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [48]:
c2008_sn = coleccion2008.map(Cloud20)
NDVI_2008_sn = c2008_sn.qualityMosaic('B11')
NDWI_2008_sn = c2008_sn.qualityMosaic('NDWI')
SAVI_2008_sn = c2008_sn.qualityMosaic('SAVI')
NDGI_2008_sn = c2008_sn.qualityMosaic('NDGI')
NDSI_2008_sn = c2008_sn.qualityMosaic('NDSI')

In [49]:
# Map2.addLayer(NDVI_2008_sn, vizParams, 'QMsn de NDVI 2008')
# Map2.addLayer(NDWI_2008_sn, vizParams, 'QMsn de NDWI 2008')
# Map2.addLayer(SAVI_2008_sn, vizParams, 'QMsn de SAVI 2008')
# Map2.addLayer(NDGI_2008_sn, vis_nieve, 'QMsn de NDGI 2008')
# Map2.addLayer(NDSI_2008_sn, vis_nieve, 'QMsn de NDSI 2008')
# Map2

### 2007

In [50]:
start_date = '2006-10-01'
end_date = '2007-03-31'

l5_2007 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [51]:
coleccion2007 = l5_2007.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [52]:
QM_NDVI_2007 = coleccion2007.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2007 = coleccion2007.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2007 = coleccion2007.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2007 = coleccion2007.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2007 = coleccion2007.qualityMosaic('NDSI') # Normalized difference snow index

In [53]:
Map.addLayer(QM_NDVI_2007, vizParams, 'QM de NDVI 2007')
Map.addLayer(QM_NDWI_2007, vizParams, 'QM de NDWI 2007')
Map.addLayer(QM_SAVI_2007, vizParams, 'QM de SAVI 2007')
Map.addLayer(QM_NDGI_2007, vis_nieve, 'QM de NDGI 2007')
Map.addLayer(QM_NDSI_2007, vis_nieve, 'QM de NDSI 2007')
# Map3

In [54]:
c2007_sn = coleccion2007.map(Cloud20)
NDVI_2007_sn = c2007_sn.qualityMosaic('B11')
NDWI_2007_sn = c2007_sn.qualityMosaic('NDWI')
SAVI_2007_sn = c2007_sn.qualityMosaic('SAVI')
NDGI_2007_sn = c2007_sn.qualityMosaic('NDGI')
NDSI_2007_sn = c2007_sn.qualityMosaic('NDSI')

In [55]:
Map2.addLayer(NDVI_2007_sn, vizParams, 'QMsn de NDVI 2007')
Map2.addLayer(NDWI_2007_sn, vizParams, 'QMsn de NDWI 2007')
Map2.addLayer(SAVI_2007_sn, vizParams, 'QMsn de SAVI 2007')
Map2.addLayer(NDGI_2007_sn, vis_nieve, 'QMsn de NDGI 2007')
Map2.addLayer(NDSI_2007_sn, vis_nieve, 'QMsn de NDSI 2007')
Map2

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2006

In [56]:
start_date = '2005-10-01'
end_date = '2006-03-31'

l5_2006 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [57]:
coleccion2006 = l5_2006.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [58]:
QM_NDVI_2006 = coleccion2006.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2006 = coleccion2006.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2006 = coleccion2006.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2006 = coleccion2006.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2006 = coleccion2006.qualityMosaic('NDSI') # Normalized difference snow index

In [59]:
Map3.addLayer(QM_NDVI_2006, vizParams, 'QM de NDVI 2006')
Map3.addLayer(QM_NDWI_2006, vizParams, 'QM de NDWI 2006')
Map3.addLayer(QM_SAVI_2006, vizParams, 'QM de SAVI 2006')
Map3.addLayer(QM_NDGI_2006, vis_nieve, 'QM de NDGI 2006')
Map3.addLayer(QM_NDSI_2006, vis_nieve, 'QM de NDSI 2006')
# Map3

In [60]:
c2006_sn = coleccion2006.map(Cloud20)
NDVI_2006_sn = c2006_sn.qualityMosaic('B11')
NDWI_2006_sn = c2006_sn.qualityMosaic('NDWI')
SAVI_2006_sn = c2006_sn.qualityMosaic('SAVI')
NDGI_2006_sn = c2006_sn.qualityMosaic('NDGI')
NDSI_2006_sn = c2006_sn.qualityMosaic('NDSI')

In [61]:
Map4.addLayer(NDVI_2006_sn, vizParams, 'QMsn de NDVI 2006')
Map4.addLayer(NDWI_2006_sn, vizParams, 'QMsn de NDWI 2006')
Map4.addLayer(SAVI_2006_sn, vizParams, 'QMsn de SAVI 2006')
Map4.addLayer(NDGI_2006_sn, vis_nieve, 'QMsn de NDGI 2006')
Map4.addLayer(NDSI_2006_sn, vis_nieve, 'QMsn de NDSI 2006')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2005

In [62]:
start_date = '2004-10-01'
end_date = '2005-03-31'

l5_2005 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [63]:
coleccion2005 = l5_2005.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [64]:
QM_NDVI_2005 = coleccion2005.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2005 = coleccion2005.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2005 = coleccion2005.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2005 = coleccion2005.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2005 = coleccion2005.qualityMosaic('NDSI') # Normalized difference snow index

In [65]:
Map3.addLayer(QM_NDVI_2005, vizParams, 'QM de NDVI 2005')
Map3.addLayer(QM_NDWI_2005, vizParams, 'QM de NDWI 2005')
Map3.addLayer(QM_SAVI_2005, vizParams, 'QM de SAVI 2005')
Map3.addLayer(QM_NDGI_2005, vis_nieve, 'QM de NDGI 2005')
Map3.addLayer(QM_NDSI_2005, vis_nieve, 'QM de NDSI 2005')
# Map3

In [66]:
c2005_sn = coleccion2005.map(Cloud20)
NDVI_2005_sn = c2005_sn.qualityMosaic('B11')
NDWI_2005_sn = c2005_sn.qualityMosaic('NDWI')
SAVI_2005_sn = c2005_sn.qualityMosaic('SAVI')
NDGI_2005_sn = c2005_sn.qualityMosaic('NDGI')
NDSI_2005_sn = c2005_sn.qualityMosaic('NDSI')

In [67]:
Map4.addLayer(NDVI_2005_sn, vizParams, 'QMsn de NDVI 2005')
Map4.addLayer(NDWI_2005_sn, vizParams, 'QMsn de NDWI 2005')
Map4.addLayer(SAVI_2005_sn, vizParams, 'QMsn de SAVI 2005')
Map4.addLayer(NDGI_2005_sn, vis_nieve, 'QMsn de NDGI 2005')
Map4.addLayer(NDSI_2005_sn, vis_nieve, 'QMsn de NDSI 2005')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2009 L7

In [68]:
start_date = '2008-10-01'
end_date = '2009-03-31'

l7_2009 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [69]:
coleccion_l7_2009 = l7_2009.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [70]:
coleccion_l7_2009 = coleccion_l7_2009.set('SENSOR_ID', 'OLI_TIRS')

In [71]:
QM_NDVI_2009_l7 = coleccion_l7_2009.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2009_l7 = coleccion_l7_2009.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2009_l7 = coleccion_l7_2009.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2009_l7 = coleccion_l7_2009.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2009_l7 = coleccion_l7_2009.qualityMosaic('NDSI') # Normalized difference snow index

In [72]:
Map5.addLayer(QM_NDVI_2009_l7, vizParams, 'QM de NDVI 2009_l7')
Map5.addLayer(QM_NDWI_2009_l7, vizParams, 'QM de NDWI 2009_l7')
Map5.addLayer(QM_SAVI_2009_l7, vizParams, 'QM de SAVI 2009_l7')
Map5.addLayer(QM_NDGI_2009_l7, vis_nieve, 'QM de NDGI 2009_l7')
Map5.addLayer(QM_NDSI_2009_l7, vis_nieve, 'QM de NDSI 2009_l7')
# Map5

In [73]:
c2009_l7_sn = coleccion_l7_2009.map(Cloud20)
NDVI_2009_l7_sn = c2009_l7_sn.qualityMosaic('NDVI')
NDWI_2009_l7_sn = c2009_l7_sn.qualityMosaic('NDWI')
SAVI_2009_l7_sn = c2009_l7_sn.qualityMosaic('SAVI')
NDGI_2009_l7_sn = c2009_l7_sn.qualityMosaic('NDGI')
NDSI_2009_l7_sn = c2009_l7_sn.qualityMosaic('NDSI')

In [74]:
Map6.addLayer(NDVI_2009_l7_sn, vizParams, 'QMsn de NDVI 2009_l7')
Map6.addLayer(NDWI_2009_l7_sn, vizParams, 'QMsn de NDWI 2009_l7')
Map6.addLayer(SAVI_2009_l7_sn, vizParams, 'QMsn de SAVI 2009_l7')
Map6.addLayer(NDGI_2009_l7_sn, vis_nieve, 'QMsn de NDGI 2009_l7')
Map6.addLayer(NDSI_2009_l7_sn, vis_nieve, 'QMsn de NDSI 2009_l7')
Map6

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2008 L7

In [75]:
start_date = '2007-10-01'
end_date = '2008-03-31'

l7_2008 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [76]:
coleccion_l7_2008 = l7_2008.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [77]:
coleccion_l7_2008 = coleccion_l7_2008.set('SENSOR_ID', 'OLI_TIRS')

In [78]:
QM_NDVI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2008_l7 = coleccion_l7_2008.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDSI') # Normalized difference snow index

In [79]:
Map5.addLayer(QM_NDVI_2008_l7, vizParams, 'QM de NDVI 2008_l7')
Map5.addLayer(QM_NDWI_2008_l7, vizParams, 'QM de NDWI 2008_l7')
Map5.addLayer(QM_SAVI_2008_l7, vizParams, 'QM de SAVI 2008_l7')
Map5.addLayer(QM_NDGI_2008_l7, vis_nieve, 'QM de NDGI 2008_l7')
Map5.addLayer(QM_NDSI_2008_l7, vis_nieve, 'QM de NDSI 2008_l7')
Map5

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [80]:
c2008_l7_sn = coleccion_l7_2008.map(Cloud20)
NDVI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDVI')
NDWI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDWI')
SAVI_2008_l7_sn = c2008_l7_sn.qualityMosaic('SAVI')
NDGI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDGI')
NDSI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDSI')

In [81]:
Map6.addLayer(NDVI_2008_l7_sn, vizParams, 'QMsn de NDVI 2008_l7')
Map6.addLayer(NDWI_2008_l7_sn, vizParams, 'QMsn de NDWI 2008_l7')
Map6.addLayer(SAVI_2008_l7_sn, vizParams, 'QMsn de SAVI 2008_l7')
Map6.addLayer(NDGI_2008_l7_sn, vis_nieve, 'QMsn de NDGI 2008_l7')
Map6.addLayer(NDSI_2008_l7_sn, vis_nieve, 'QMsn de NDSI 2008_l7')
Map6

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2007 L7

In [82]:
start_date = '2006-10-01'
end_date = '2007-03-31'

l7_2007 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [83]:
coleccion_l7_2007 = l7_2007.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [84]:
QM_NDVI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2007_l7 = coleccion_l7_2007.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDSI') # Normalized difference snow index

In [85]:
Map5.addLayer(QM_NDVI_2007_l7, vizParams, 'QM de NDVI 2007_l7')
Map5.addLayer(QM_NDWI_2007_l7, vizParams, 'QM de NDWI 2007_l7')
Map5.addLayer(QM_SAVI_2007_l7, vizParams, 'QM de SAVI 2007_l7')
Map5.addLayer(QM_NDGI_2007_l7, vis_nieve, 'QM de NDGI 2007_l7')
Map5.addLayer(QM_NDSI_2007_l7, vis_nieve, 'QM de NDSI 2007_l7')
# Map5

In [86]:
c2007_l7_sn = coleccion_l7_2007.map(Cloud20)
NDVI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDVI')
NDWI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDWI')
SAVI_2007_l7_sn = c2007_l7_sn.qualityMosaic('SAVI')
NDGI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDGI')
NDSI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDSI')

In [87]:
Map6.addLayer(NDVI_2007_l7_sn, vizParams, 'QMsn de NDVI 2007_l7')
Map6.addLayer(NDWI_2007_l7_sn, vizParams, 'QMsn de NDWI 2007_l7')
Map6.addLayer(SAVI_2007_l7_sn, vizParams, 'QMsn de SAVI 2007_l7')
Map6.addLayer(NDGI_2007_l7_sn, vis_nieve, 'QMsn de NDGI 2007_l7')
Map6.addLayer(NDSI_2007_l7_sn, vis_nieve, 'QMsn de NDSI 2007_l7')
Map6

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2006 L7

In [88]:
start_date = '2005-10-01'
end_date = '2006-03-31'

l7_2006 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [89]:
coleccion_l7_2006 = l7_2006.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [90]:
QM_NDVI_2006_l7 = coleccion_l7_2006.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2006_l7 = coleccion_l7_2006.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2006_l7 = coleccion_l7_2006.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2006_l7 = coleccion_l7_2006.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2006_l7 = coleccion_l7_2006.qualityMosaic('NDSI') # Normalized difference snow index

In [91]:
Map5.addLayer(QM_NDVI_2006_l7, vizParams, 'QM de NDVI 2006_l7')
Map5.addLayer(QM_NDWI_2006_l7, vizParams, 'QM de NDWI 2006_l7')
Map5.addLayer(QM_SAVI_2006_l7, vizParams, 'QM de SAVI 2006_l7')
Map5.addLayer(QM_NDGI_2006_l7, vis_nieve, 'QM de NDGI 2006_l7')
Map5.addLayer(QM_NDSI_2006_l7, vis_nieve, 'QM de NDSI 2006_l7')
# Map5

In [92]:
c2006_l7_sn = coleccion_l7_2006.map(Cloud20)
NDVI_2006_l7_sn = c2006_l7_sn.qualityMosaic('NDVI')
NDWI_2006_l7_sn = c2006_l7_sn.qualityMosaic('NDWI')
SAVI_2006_l7_sn = c2006_l7_sn.qualityMosaic('SAVI')
NDGI_2006_l7_sn = c2006_l7_sn.qualityMosaic('NDGI')
NDSI_2006_l7_sn = c2006_l7_sn.qualityMosaic('NDSI')

In [93]:
# Map8.addLayer(NDVI_2006_l7_sn, vizParams, 'QMsn de NDVI 2006_l7')
# Map8.addLayer(NDWI_2006_l7_sn, vizParams, 'QMsn de NDWI 2006_l7')
# Map8.addLayer(SAVI_2006_l7_sn, vizParams, 'QMsn de SAVI 2006_l7')
# Map8.addLayer(NDGI_2006_l7_sn, vis_nieve, 'QMsn de NDGI 2006_l7')
# Map8.addLayer(NDSI_2006_l7_sn, vis_nieve, 'QMsn de NDSI 2006_l7')
# Map8

### 2006-2009 Colección

In [94]:
coleccion2006_2009 = ee.ImageCollection(l7_2009.merge(l7_2008).merge(l7_2007).merge(l7_2006))

In [95]:
coleccion2006_2009_l5 = ee.ImageCollection(l5_2009.merge(l5_2008).merge(l5_2007).merge(l5_2006).merge(l5_2005))

In [96]:
l7_summer = coleccion2006_2009.map(addNDVI7).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [97]:
l5_summer = coleccion2006_2009_l5.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

### Tier 1

In [98]:
Tier1_l5_sn = l5_summer.map(Cloud10)

In [99]:
QM_NDVI_T1_l5 = Tier1_l5_sn.qualityMosaic('B11')# Normalized difference vegetation Index
QM_NDWI_T1_l5 = Tier1_l5_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T1_l5 = Tier1_l5_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T1_l5 = Tier1_l5_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T1_l5 = Tier1_l5_sn.qualityMosaic('NDSI') # Normalized difference snow index

In [100]:
T_median = Tier1_l5_sn.median()

In [101]:
Map2.addLayer(QM_NDVI_T1_l5, vizParams, 'QM de NDVI T1 L5')
Map2.addLayer(QM_NDWI_T1_l5, vizParams, 'QM de NDWI T1 L5')
Map2.addLayer(T_median, vizParams, 'mediana')
Map2 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [102]:
Tier1_sn = l7_summer.map(Cloud10)

In [103]:
QM_NDVI_T1 = Tier1_sn.qualityMosaic('NDVI')# Normalized difference vegetation Index
QM_NDWI_T1 = Tier1_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T1 = Tier1_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T1 = Tier1_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T1 = Tier1_sn.qualityMosaic('NDSI') # Normalized difference snow index

In [104]:
Map2.addLayer(QM_NDVI_T1, vizParams, 'QM de NDVI T1')
# Map2.addLayer(QM_NDWI_T1, vizParams, 'QM de NDWI T1')
# Map2.addLayer(QM_SAVI_T1, vizParams, 'QM de SAVI T1')
# Map2.addLayer(QM_NDGI_T1, vis_nieve, 'QM de NDGI T1')
# Map2.addLayer(QM_NDSI_T1, vis_nieve, 'QM de NDSI T1')
Map2 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Tier 2

In [105]:
Tier2_l5_sn = l5_summer.map(Cloud20)

In [106]:
QM_NDVI_T2_l5 = Tier2_l5_sn.qualityMosaic('B11')# Normalized difference vegetation Index
QM_NDWI_T2_l5 = Tier2_l5_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T2_l5 = Tier2_l5_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T2_l5 = Tier2_l5_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T2_l5 = Tier2_l5_sn.qualityMosaic('NDSI') # Normalized difference snow index

In [107]:
T2_median = Tier2_l5_sn.median()

In [108]:
Tier2_sn = l7_summer.map(Cloud20)

In [109]:
QM_NDVI_T2 = Tier2_sn.qualityMosaic('NDVI')# Normalized difference vegetation Index
QM_NDWI_T2 = Tier2_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T2 = Tier2_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T2 = Tier2_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T2 = Tier2_sn.qualityMosaic('NDSI') # Normalized difference snow index

### Tier 3

In [110]:
Tier3_l5_sn = l5_summer.map(Cloud30)

In [111]:
QM_NDVI_T3_l5 = Tier3_l5_sn.qualityMosaic('B11')# Normalized difference vegetation Index
QM_NDWI_T3_l5 = Tier3_l5_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T3_l5 = Tier3_l5_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T3_l5 = Tier3_l5_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T3_l5 = Tier3_l5_sn.qualityMosaic('NDSI') # Normalized difference snow index

In [112]:
T3_median = Tier3_l5_sn.median()

In [113]:
Tier3_sn = l7_summer.map(Cloud30)

In [114]:
QM_NDVI_T3 = Tier3_sn.qualityMosaic('NDVI')# Normalized difference vegetation Index
QM_NDWI_T3 = Tier3_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T3 = Tier3_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T3 = Tier3_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T3 = Tier3_sn.qualityMosaic('NDSI') # Normalized difference snow index

### Tier 4

In [115]:
Tier4_l5_sn = l5_summer.map(Cloud40)

In [116]:
QM_NDVI_T4_l5 = Tier4_l5_sn.qualityMosaic('B11')# Normalized difference vegetation Index
QM_NDWI_T4_l5 = Tier4_l5_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T4_l5 = Tier4_l5_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T4_l5 = Tier4_l5_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T4_l5 = Tier4_l5_sn.qualityMosaic('NDSI') # Normalized difference snow index

In [117]:
T4_median = Tier4_l5_sn.median()

In [118]:
Tier4_sn = l7_summer.map(Cloud40)

In [119]:
QM_NDVI_T4 = Tier4_sn.qualityMosaic('NDVI')# Normalized difference vegetation Index
QM_NDWI_T4 = Tier4_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T4 = Tier4_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T4 = Tier4_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T4 = Tier4_sn.qualityMosaic('NDSI') # Normalized difference snow index

In [120]:
T4_median_l7 = Tier4_sn.median()

In [121]:
Map2.addLayer(T4_median, vizParams, 'mediana4')
Map2.addLayer(T4_median_l7, vizParams, 'mediana4 l7')
Map2

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Img base

In [122]:
start_date = '2008-10-01'
end_date = '2009-03-31'
imgbase = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(base) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first() 


In [123]:
# imgbase.get('LANDSAT_PRODUCT_ID').getInfo()

In [124]:
# imgbase = ee.Image('LANDSAT/LE07/C01/T1_TOA/LE07_230092_20081213')

In [125]:
# Map.addLayer(colec_base, vizParams,'1')
# Map

In [126]:
start_date = '2008-10-01'
end_date = '2009-03-31'
imgbase2 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 230)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [127]:
imgbase2 = find_closest(imgbase, imgbase2,14).sort('CLOUD_COVER').first()
imgbase2 = ee.Image(imgbase2)

In [128]:
# Map1.addLayer(imgbase2, vizParams, 'img base2')
# Map1

In [129]:
start_date = '2008-10-01'
end_date = '2009-03-31'
imgbase3 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 229)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [130]:
imgbase3 = find_closest(imgbase2, imgbase3,21).sort('CLOUD_COVER').first()
imgbase3 = ee.Image(imgbase3)

In [131]:
start_date = '2008-10-01'
end_date = '2009-03-31'
imgbase4 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 228)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [132]:
imgbase4 = find_closest(imgbase3, imgbase4,14).sort('CLOUD_COVER').first()
imgbase4 = ee.Image(imgbase4)

In [133]:
start_date = '2008-10-01'
end_date = '2009-03-31'
imgbase5 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 232)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [134]:
imgbase5 = find_closest(imgbase, imgbase5,28).sort('CLOUD_COVER').first()
imgbase5 = ee.Image(imgbase5)

In [135]:
start_date = '2008-10-01'
end_date = '2009-03-31'
imgbase6 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 233)) \
    .filter(ee.Filter.eq('WRS_ROW', 91)) \
    .filterDate(start_date, end_date) 

In [136]:
imgbase6 = find_closest(imgbase5, imgbase6,360).sort('CLOUD_COVER').first()
imgbase6 = ee.Image(imgbase6)

In [137]:
colec_base = ee.ImageCollection([imgbase,imgbase2,imgbase3,imgbase4,imgbase5,imgbase6]).mosaic()

In [138]:

# Map1.addLayer(imgbase, vizParams, 'img base')
# Map1.addLayer(imgbase2, vizParams, 'img base2')
# Map1.addLayer(imgbase3, vizParams, 'img base3')
# Map1.addLayer(imgbase4, vizParams, 'img base4')
# Map1.addLayer(imgbase5, vizParams, 'img base5')
# Map1.addLayer(imgbase6, vizParams, 'img base6')
# # Map1.addLayer(colec_base, vizParams, 'coleccion')
# # Map1.addLayer(colec_base_r, vizParams, 'coleccion r')
# Map1

## Indices por colección

### Índices 2008

In [139]:
NDVI_2008 = NDVI_2008_sn.select('B11')
NDWI_2008 = NDWI_2008_sn.select('NDWI') 
SAVI_2008 = SAVI_2008_sn.select('SAVI')
NDGI_2008 = NDGI_2008_sn.select('NDGI')
NDSI_2008 = NDSI_2008_sn.select('NDSI')

In [140]:
# Map7.addLayer(NDVI_2008, {'palette': color_terrain}, 'QMsn de NDVI 2008')
# Map7.addLayer(NDWI_2008, {'palette': color_ndwi}, 'QMsn de NDWI 2008')
# Map7.addLayer(SAVI_2008, {'palette': color_ndvi}, 'QMsn de SAVI 2008')
# # Map7.addLayer(NDGI_2008, {'palette': color_glaciar}, 'QMsn de NDGI 2008')
# # Map7.addLayer(NDSI_2008, {'palette': color_ndwi}, 'QMsn de NDSI 2008')
# Map7

### Índices 2007

In [141]:
NDVI_2007 = NDVI_2007_sn.select('B11')
NDWI_2007 = NDWI_2007_sn.select('NDWI') 
SAVI_2007 = SAVI_2007_sn.select('SAVI')
NDGI_2007 = NDGI_2007_sn.select('NDGI')
NDSI_2007 = NDSI_2007_sn.select('NDSI')

In [142]:
# Map7.addLayer(NDVI_2007, {'palette': color_terrain}, 'QMsn de NDVI 2007')
# Map7.addLayer(NDWI_2007, {'palette': color_ndwi}, 'QMsn de NDWI 2007')
# Map7.addLayer(SAVI_2007, {'palette': color_ndvi}, 'QMsn de SAVI 2007')
# # Map7.addLayer(NDGI_2007, {'palette': color_glaciar}, 'QMsn de NDGI 2007')
# # Map7.addLayer(NDSI_2007, {'palette': color_ndwi}, 'QMsn de NDSI 2007')
# Map7

### Índices 2006

In [143]:
NDVI_2006 = NDVI_2006_sn.select('B11')
NDWI_2006 = NDWI_2006_sn.select('NDWI') 
SAVI_2006 = SAVI_2006_sn.select('SAVI')
NDGI_2006 = NDGI_2006_sn.select('NDGI')
NDSI_2006 = NDSI_2006_sn.select('NDSI')

In [144]:
# Map8.addLayer(NDVI_2006, {'palette': color_terrain}, 'QMsn de NDVI 2006')
# Map8.addLayer(NDWI_2006, {'palette': color_ndwi}, 'QMsn de NDWI 2006')
# Map8.addLayer(SAVI_2006, {'palette': color_ndvi}, 'QMsn de SAVI 2006')
# # Map8.addLayer(NDGI_2006, {'palette': color_glaciar}, 'QMsn de NDGI 2006')
# # Map8.addLayer(NDSI_2006, {'palette': color_ndwi}, 'QMsn de NDSI 2006')
# Map8

### Índices 2005 

In [145]:
NDVI_2005 = NDVI_2005_sn.select('B11')
NDWI_2005 = NDWI_2005_sn.select('NDWI') 
SAVI_2005 = SAVI_2005_sn.select('SAVI')
NDGI_2005 = NDGI_2005_sn.select('NDGI')
NDSI_2005 = NDSI_2005_sn.select('NDSI')

In [146]:
# Map8.addLayer(NDVI_2005, {'palette': color_terrain}, 'QMsn de NDVI 2005')
# Map8.addLayer(NDWI_2005, {'palette': color_ndwi}, 'QMsn de NDWI 2005')
# Map8.addLayer(SAVI_2005, {'palette': color_ndvi}, 'QMsn de SAVI 2005')
# # Map8.addLayer(NDGI_2005, {'palette': color_glaciar}, 'QMsn de NDGI 2005')
# # Map8.addLayer(NDSI_2005, {'palette': color_ndwi}, 'QMsn de NDSI 2005')
# Map8

### Índices 2008 L7

In [147]:
NDVI_2008_l7 = NDVI_2008_l7_sn.select('NDVI')
NDWI_2008_l7 = NDWI_2008_l7_sn.select('NDWI')
SAVI_2008_l7 = SAVI_2008_l7_sn.select('SAVI')
NDGI_2008_l7 = NDGI_2008_l7_sn.select('NDGI')
NDSI_2008_l7 = NDSI_2008_l7_sn.select('NDSI')

In [148]:
# Map8.addLayer(NDVI_2008_l7, {'palette': color_terrain}, 'QMsn de NDVI 2008_l7')
# Map8.addLayer(NDWI_2008_l7, {'palette': color_ndwi}, 'QMsn de NDWI 2008_l7')
# Map8.addLayer(SAVI_2008_l7, {'palette': color_ndvi}, 'QMsn de SAVI 2008_l7')
# # Map8.addLayer(NDGI_2008_l7, {'palette': color_glaciar}, 'QMsn de NDGI 2008_l7')
# # Map8.addLayer(NDSI_2008_l7, {'palette': color_ndwi}, 'QMsn de NDSI 2008_l7')
# Map8

## Generación de Mosaico L7

In [149]:
# mosaic = ee.ImageCollection([QM_NDGI_2009_l7,NDVI_2008_l7_sn]).mosaic()

In [150]:
NDVI_landsat_7 = QM_NDVI_2008_l7.set('SENSOR_ID', 'ETM+')

In [151]:
cloud_ = ee.Algorithms.Landsat.simpleCloudScore(NDVI_landsat_7)

In [152]:
cloud_ = cloud_.select('cloud')

In [153]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [154]:
resultados= [cloud_,NDVI_landsat_7]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes',
    'QM NDVI']

In [155]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [156]:
def llenado(image):
    return image.where(cloud_.gt(10),QM_NDVI_T1)

In [157]:
img1 = llenado(NDVI_landsat_7)

In [158]:
img1_landsat = img1.set('SENSOR_ID', 'ETM+')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [159]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [160]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [161]:
def llenado2(image):
    return image.where(cloud_img1.gt(10), QM_NDVI_T2)

In [162]:
img2 = llenado2(img1)

In [163]:
img2_landsat = img2.set('SENSOR_ID', 'ETM+')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [164]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [165]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [166]:
def llenado3(image):
    return image.where(cloud_img2.gt(10), QM_NDVI_T3)

In [167]:
img3 = llenado3(img2)

In [168]:
img3_landsat = img3.set('SENSOR_ID', 'ETM+')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [169]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [170]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [171]:
def llenado4(image):
    return image.where(cloud_img3.gt(10), QM_NDVI_T4)

In [172]:
img4 = llenado4(img3)

In [173]:
img4_landsat = img4.set('SENSOR_ID', 'ETM+')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [174]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [175]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [176]:
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')
datamask = hansenImage.select('datamask')

In [177]:
def llenado5(image):
    return image.where(datamask.eq(2), QM_NDWI_T1)

In [178]:
img5 = llenado5(img4)

In [179]:
resultados= [img4, img5]
labels = [
    'Mosaico 4',
    'Mosaico 5']
vis_params3=  [
     vizParams, 
    vizParams]

In [180]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [181]:
Map10.addLayer(img4, vizParams, 'Mosaico 4')
Map10.addLayer(img5, vizParams, 'Mosaico 5')
Map10

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Generación Mosaico L5

In [182]:
# mosaic_landsat = coleccion2007.median()

In [183]:
mosaic_landsat = T4_median.set('SENSOR_ID', 'OLI_TIRS')

In [184]:
cloud_mosaic = ee.Algorithms.Landsat.simpleCloudScore(mosaic_landsat)

In [185]:
cloud_mosaic = cloud_mosaic.select('cloud')

In [186]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [187]:
resultados= [cloud_mosaic, T4_median]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes mosaico',
    'mosaico inicial']

In [188]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [189]:
def llenado(image):
    return image.where(cloud_mosaic.gt(10),T_median)

In [190]:
img1 = llenado(T4_median)

In [191]:
img1_landsat = img1.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [192]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [193]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [194]:
def llenado2(image):
    return image.where(cloud_img1.gt(10), T2_median)

In [195]:
img2 = llenado2(img1)

In [196]:
img2_landsat = img2.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [197]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [198]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [199]:
def llenado3(image):
    return image.where(cloud_img2.gt(10), T3_median)

In [200]:
img3 = llenado3(img2)

In [201]:
img3_landsat = img3.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [202]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [203]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [204]:
def llenado4(image):
    return image.where(cloud_img3.gt(10), T4_median)

In [205]:
img4 = llenado4(img3)

In [206]:
img4_landsat = img4.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [207]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [208]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [209]:
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')
datamask = hansenImage.select('datamask')

In [210]:
def llenado5(image):
    return image.where(datamask.eq(2), QM_NDWI_T1_l5)

In [211]:
img5 = llenado5(img4)

In [212]:
img5_landsat = img5.set('SENSOR_ID', 'OLI_TIRS')
cloud_img5 = ee.Algorithms.Landsat.simpleCloudScore(img5_landsat)
cloud_img5 = cloud_img5.select('cloud')

In [213]:
resultados= [img4, img5]
labels = [
    'Mosaico 4',
    'Mosaico 5']
vis_params3=  [
     vizParams, 
    vizParams]

In [214]:
Map10.addLayer(img4, vizParams, 'Mosaico 4')
Map10.addLayer(img5, vizParams, 'Mosaico 5')
Map10

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [215]:
img5_b = img5.select(['B1','B2','B3','B4','B5','B6','B7'])

In [216]:
molde = os.path.join(work_dir, 'molde.shp')
molde_grande = geemap.shp_to_ee(molde)

In [217]:
# img5_b_c = img5_b.clip(molde_grande).unmask()

## Exportación

In [218]:
img4_b = img4.select(['B1','B2','B3','B4','B5','B6','B7'])

In [219]:
partes_1 = os.path.join(work_dir, '1.shp')
partes1 = geemap.shp_to_ee(partes_1)

partes_2 = os.path.join(work_dir, '2.shp')
partes2 = geemap.shp_to_ee(partes_2)

partes_3 = os.path.join(work_dir, '3.shp')
partes3 = geemap.shp_to_ee(partes_3)

partes_4 = os.path.join(work_dir, '4.shp')
partes4 = geemap.shp_to_ee(partes_4)

partes_5 = os.path.join(work_dir, '5.shp')
partes5 = geemap.shp_to_ee(partes_5)

partes_6 = os.path.join(work_dir, '6.shp')
partes6 = geemap.shp_to_ee(partes_6)

partes_7 = os.path.join(work_dir, '7.shp')
partes7 = geemap.shp_to_ee(partes_7)

partes_8 = os.path.join(work_dir, '8.shp')
partes8 = geemap.shp_to_ee(partes_8)

In [220]:
# reproj = img5_b \
#     .reproject(crs='EPSG:32719',scale = 30)

In [221]:
geometry_exp = ee.Geometry.Polygon(
        [[[-72.56281773604495, -33.81999514196144],
          [-72.96381871260745, -35.294382843203024],
          [-73.27692906416995, -35.25850698608905],
          [-74.15583531416995, -38.13757615203617],
          [-73.96357457198245, -38.16349478321529],
          [-74.41603250978783, -39.59214901926122],
          [-73.71840067385033, -39.70212049569608],
          [-74.48057965994737, -41.961233926702086],
          [-76.23199334221133, -46.599308425949715],
          [-75.71838250236758, -46.6705186900265],
          [-76.31583416780423, -48.08335993023956],
          [-75.6739504332226, -48.17601084778724],
          [-76.1812028986966, -49.3089458997328],
          [-75.8516130549466, -49.35190435466257],
          [-75.88182545729035, -49.42252520588146],
          [-75.62776661939972, -49.4593908439046],
          [-75.87838309772656, -50.0286830263256],
          [-75.4746355391328, -50.08951678614621],
          [-75.54325576346766, -50.23909657453096],
          [-75.77780697097086, -50.76007049594944],
          [-75.16119930495523, -50.84458853545005],
          [-75.38504574050211, -51.344039508567974],
          [-75.5541538866841, -51.71140521484993],
          [-75.18336531246536, -51.75648162278235],
          [-75.47788882049618, -52.3659753295982],
          [-74.75894718310815, -52.46884703800526],
          [-75.08565455837835, -53.06208069103826],
          [-74.55801479059262, -53.126414996444055],
          [-74.81690883977036, -53.6371915734125],
          [-73.44408000172288, -53.83741753537409],
          [-73.50862467945726, -53.95798563315933],
          [-73.45231974781663, -53.967680681913386],
          [-73.53197062672288, -54.11766576333605],
          [-73.77219189508037, -54.57485104154676],
          [-72.34049166499341, -54.77775087176573],
          [-72.47759402717024, -55.03231503944354],
          [-71.05285526571967, -55.24340674895388],
          [-71.08306766806342, -55.29113504937561],
          [-69.81960741798575, -55.47109132991434],
          [-69.85119311134513, -55.51932330880519],
          [-69.68227831642325, -55.544194128455445],
          [-69.79214159767325, -55.75341614311607],
          [-66.85410518726452, -56.21137383375629],
          [-66.30968752348785, -55.25322454720416],
          [-65.44176760161285, -55.378260612328376],
          [-65.29222191220039, -55.084940556988364],
          [-63.8145607793879, -55.29503930747886],
          [-63.4410256231379, -54.559610949790994],
          [-66.01731956845039, -54.18844646199444],
          [-65.92942894345039, -53.97576673331468],
          [-67.35765159970039, -53.77173637481835],
          [-66.83490788218774, -52.56216102836118],
          [-68.29059635875024, -52.351280557744616],
          [-67.13664467133525, -49.5665018755825],
          [-65.6891959408665, -49.77447361491692],
          [-64.6040407061826, -46.886773053508534],
          [-66.07346209290135, -46.670468623465915],
          [-65.60651222417681, -45.29237148760689],
          [-64.13159767339556, -45.51413819831776],
          [-63.11762183127383, -42.44210625114028],
          [-61.64820044455508, -42.65254570178212],
          [-60.691700635610566, -39.58656313087865],
          [-60.612049756704316, -39.597145764356114],
          [-60.214053037778356, -38.26345264755797],
          [-62.338430585556544, -37.86345563921212],
          [-62.431814374619044, -38.13616213643378],
          [-63.880475463562526, -37.8656239671591],
          [-63.965619506531276, -38.12752037872568],
          [-65.43863114657343, -37.86678770192114],
          [-65.52652177157343, -38.12867996539086],
          [-66.31753739657343, -37.98811028708228],
          [-65.98609312949165, -36.82548737245019],
          [-68.06866521074468, -36.43149799324939],
          [-68.16204899980718, -36.70723234878123],
          [-69.00250310136968, -36.55294237984414],
          [-68.66838285254276, -35.38834808765389],
          [-70.73190167990279, -35.008946233837705],
          [-70.82528546896529, -35.29190694633577],
          [-71.68771222677779, -35.11685679848462],
          [-71.37460187521529, -33.95679291780399],
          [-72.53365949240279, -33.733238932504804]]])

In [222]:
img5.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'BQA',
 'B11',
 'NDWI',
 'NDGI',
 'SAVI',
 'NDSI',
 'date',
 'nube']

In [223]:
img5_B1 = img5.select("B1")
img5_B2 = img5.select("B2")
img5_B3 = img5.select("B3")
img5_B4 = img5.select("B4")
img5_B5 = img5.select("B5")
img5_B6 = img5.select("B6")
img5_B7 = img5.select("B7")

In [224]:
task1 = ee.batch.Export.image.toDrive(**{
    'image': img5_B1,
    'description':'B1',
    'folder':'exp2021_19s_full',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task2 = ee.batch.Export.image.toDrive(**{
    'image': img5_B2,
    'description':'B2',
    'folder':'exp2021_19s_full',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task3 = ee.batch.Export.image.toDrive(**{
    'image': img5_B3,
    'description':'B3',
    'folder':'exp2021_19s_full',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task4 = ee.batch.Export.image.toDrive(**{
    'image': img5_B4,
    'description':'B4',
    'folder':'exp2021_19s_full',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task5 = ee.batch.Export.image.toDrive(**{
    'image': img5_B5,
    'description':'B5',
    'folder':'exp2021_19s_full',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task6 = ee.batch.Export.image.toDrive(**{
    'image': img5_B6,
    'description':'B6',
    'folder':'exp2021_19s_full',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})
task7 = ee.batch.Export.image.toDrive(**{
    'image': img5_B7,
    'description':'B7',
    'folder':'exp2021_19s_full',
    'region': geometry_exp.bounds(),
    'scale':30,
    'crsTransform':[30, 0, 214185, 0, -30, -4825185], 
    'crs':'EPSG:32719',
    'maxPixels':1e13
})

In [225]:
# task1.start()
# task2.start()
# task3.start()
# task4.start()
# task5.start()
# task6.start()
# task7.start()

In [226]:
# img5_1 = reproj.clip(partes1).unmask()
# img5_2 = reproj.clip(partes2).unmask()
# img5_3 = reproj.clip(partes3).unmask()
# img5_4 = reproj.clip(partes4).unmask()
# img5_5 = reproj.clip(partes5).unmask()
# img5_6 = reproj.clip(partes6).unmask()
# img5_7 = reproj.clip(partes7).unmask()
# img5_8 = reproj.clip(partes8).unmask()

In [227]:
# geemap.ee_export_image_to_drive(img4_1, description='mosaico4_2008_1', folder='exp_2008_19s', region=partes1.geometry(), scale=30)
# geemap.ee_export_image_to_drive(img4_2, description='mosaico4_2008_2', folder='exp_2008_19s', region=partes2.geometry(), scale=30)
# geemap.ee_export_image_to_drive(img4_3, description='mosaico4_2008_3', folder='exp_2008_19s', region=partes3.geometry(), scale=30)
# geemap.ee_export_image_to_drive(img4_4, description='mosaico4_2008_4', folder='exp_2008_19s', region=partes4.geometry(), scale=30)
# geemap.ee_export_image_to_drive(img4_5, description='mosaico4_2008_5', folder='exp_2008_19s', region=partes5.geometry(), scale=30)
# geemap.ee_export_image_to_drive(img4_6, description='mosaico4_2008_6', folder='exp_2008_19s', region=partes6.geometry(), scale=30)
# geemap.ee_export_image_to_drive(img4_7, description='mosaico4_2008_7', folder='exp_2008_19s', region=partes7.geometry(), scale=30)
# geemap.ee_export_image_to_drive(img4_8, description='mosaico4_2008_8', folder='exp_2008_19s', region=partes8.geometry(), scale=30)

In [228]:
# geemap.ee_export_image_to_drive(reproj1, description='mosaico_2008_1', folder='exp_2008_19s', region=partes1.geometry(), scale=30)
# geemap.ee_export_image_to_drive(reproj2, description='mosaico_2008_2', folder='exp_2008_19s', region=partes2.geometry(), scale=30)
# geemap.ee_export_image_to_drive(reproj3, description='mosaico_2008_3', folder='exp_2008_19s', region=partes3.geometry(), scale=30)
# geemap.ee_export_image_to_drive(reproj4, description='mosaico_2008_4', folder='exp_2008_19s', region=partes4.geometry(), scale=30)
# geemap.ee_export_image_to_drive(reproj5, description='mosaico_2008_5', folder='exp_2008_19s', region=partes5.geometry(), scale=30)
# geemap.ee_export_image_to_drive(reproj6, description='mosaico_2008_6', folder='exp_2008_19s', region=partes6.geometry(), scale=30)
# geemap.ee_export_image_to_drive(reproj7, description='mosaico_2008_7', folder='exp_2008_19s', region=partes7.geometry(), scale=30)
# geemap.ee_export_image_to_drive(reproj8, description='mosaico_2008_8', folder='exp_2008_19s', region=partes8.geometry(), scale=30)

In [229]:
# geemap.ee_export_image_to_drive(img5_1, description='mosaico_2008_1', folder='expR', region=partes1.geometry(), scale=30)

In [230]:
# geemap.ee_export_image_to_drive(img5_2, description='mosaico_2008_2', folder='expR', region=partes2.geometry(), scale=30)

In [231]:
# geemap.ee_export_image_to_drive(img5_3, description='mosaico_19s_3', folder='expR', region=partes3.geometry(), scale=30)

In [232]:
# geemap.ee_export_image_to_drive(img5_4, description='mosaico_2008_4', folder='expR', region=partes4.geometry(), scale=30)

In [233]:
# geemap.ee_export_image_to_drive(img5_5, description='mosaico_2008_5', folder='expR', region=partes5.geometry(), scale=30)

In [234]:
# geemap.ee_export_image_to_drive(img5_6, description='mosaico_2008_6', folder='expR', region=partes6.geometry(), scale=30)

In [235]:
# geemap.ee_export_image_to_drive(img5_7, description='mosaico_2008_7', folder='expR', region=partes7.geometry(), scale=30)

In [236]:
# geemap.ee_export_image_to_drive(img5_8, description='mosaico_2008_8', folder='expR', region=partes8.geometry(), scale=30)

In [237]:
# reproj= img5_b \
#     .reproject(crs='EPSG:4326',scale = 30)

In [238]:
# geemap.ee_export_image_to_drive(reproj, description='mosaico_2008', folder='export2', region=patagonia_grande.geometry(), scale=30)